<a href="https://www.kaggle.com/code/fareselmenshawii/random-forest-from-scratch?scriptVersionId=117153730" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div class="table-of-contents" style="background-color:#43ae38; padding: 20px; margin: 10px; font-size: 110%; border-radius: 25px; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);">
  <h1 style="color:#000000;">TOC</h1>
  <ol>
     <li><a href="#1" style="color: #000000;">1. Overview</a></li>
    <li><a href="#2" style="color: #000000;">2. Imports</a></li>
      <li><a href="#3" style="color: #000000;">3. Data Analysis</a></li>
    <li><a href="#4" style="color: #000000;">4. Data Preprocessing</a></li>
    <li><a href="#5" style="color: #000000;">5. Model Implementation</a></li>
    <li><a href="#6" style="color: #000000;">6. Evaluation</a></li>
          <li><a href="#7" style="color: #000000;">7. Conclusion</a></li>
   <li><a href="#8" style="color: #000000;">8. Thank You</a></li> 
  </ol>
</div>

<a id="1"></a>
<h1 style='background:#43ae38;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Overview</center></h1>
    
    
# Overview
**Previously we've implemented a [Decision Tree Classifier](https://www.kaggle.com/code/fareselmenshawii/decision-tree-classifier-from-scratch)**

**In this notebook we'll be building on top of that to build a Random Forest Classifier**

**Let's get started**    


<a id="2"></a>
<h1 style='background:#43ae38;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Imports</center></h1>
    

# Imports


In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

<a id="3"></a>
<h1 style='background:#43ae38;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Data Analysis</center></h1>
    

# Data Analysis


In [2]:
iris = pd.read_csv("../input/iris/Iris.csv") #Load Data
iris.drop('Id',inplace=True,axis=1) #Drop Id column

In [3]:
iris.head().style.background_gradient(cmap =sns.light_palette("seagreen", as_cmap=True)
)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.100000,3.500000,1.400000,0.200000,Iris-setosa
1,4.900000,3.000000,1.400000,0.200000,Iris-setosa
2,4.700000,3.200000,1.300000,0.200000,Iris-setosa
3,4.600000,3.100000,1.500000,0.200000,Iris-setosa
4,5.000000,3.600000,1.400000,0.200000,Iris-setosa


In [4]:
X_df = iris.iloc[:,:-1] #Set our training dataframe

y_df = iris.iloc[:,-1] # Set our training labels dataframe


In [5]:
fig = px.scatter(data_frame=iris, x='SepalLengthCm',color='Species',template='plotly_dark')
fig.update_layout(width=800, height=600,
                  xaxis=dict(title='SepalLengthCm',color="#36FF00"),
                 yaxis=dict(title="Flower Name",color="#36FF00"))
fig.show()

In [6]:
fig = px.scatter(data_frame=iris, x='SepalWidthCm',color='Species',template='plotly_dark')
fig.update_layout(width=800, height=600,
                  xaxis=dict(title='SepalWidthCm',color="#36FF00"),
                 yaxis=dict(title="Flower Name",color="#36FF00"))
fig.show()

In [7]:
fig = px.scatter(data_frame=iris, x='PetalLengthCm',color='Species',template='plotly_dark')
fig.update_layout(width=800, height=600,
                  xaxis=dict(title='PetalLengthCm',color="#36FF00"),
                 yaxis=dict(title="Flower Name",color="#36FF00"))
fig.show()

In [8]:
fig = px.scatter(data_frame=iris, x='PetalWidthCm',color='Species',template='plotly_dark')
fig.update_layout(width=800, height=600,
                  xaxis=dict(title='PetalWidthCm',color="#36FF00"),
                 yaxis=dict(title="Flower Name",color="#36FF00"))
fig.show()

In [9]:
fig = px.scatter(data_frame=iris, x='SepalLengthCm',y='SepalWidthCm',
           size='PetalLengthCm',color='Species',template='plotly_dark')

fig.update_layout(width=800, height=600,
                  xaxis=dict(title='SepalLengthCm',color="#36FF00"),
                 yaxis=dict(title="SepalWidthCm",color="#36FF00"))
fig.show()

<a id="4"></a>
<h1 style='background:#43ae38;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Data Preprocessing</center></h1>
    

# Data Preprocessing


In [10]:
from sklearn.preprocessing import LabelEncoder
labelencord = LabelEncoder()
iris['Species'] = labelencord.fit_transform(iris['Species'])

In [11]:
X = iris.iloc[:, :-1].values
y = iris.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=41)

<a id="5"></a>
<h1 style='background:#43ae38;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Model Implementation</center></h1>
    

# Model Implementation


## Decision Tree Implementation
**As mentioned before it's good idea to be famillar with [decision trees](https://www.kaggle.com/code/fareselmenshawii/decision-tree-classifier-from-scratch) to get started with random forest** 

In [12]:
class Node():
    def __init__(self, feature=None, threshold=None, left=None, right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        #If a leaf node
        self.value = value
        
        
class DecisionTree():

    def __init__(self, min_samples=2, max_depth=2):
        self.min_samples = min_samples
        self.max_depth = max_depth

        
    def split_data(self, dataset, feature, threshold):
        #create left, right datasets as empty arrays
        left_dataset = []
        right_dataset = []
        # loop over each row in the dataset
        for row in dataset:
            if row[feature] <= threshold: # compare row to threshold
                left_dataset.append(row)
            else:
                right_dataset.append(row)
        # convert left, right datasets to numpy arrays
        left_dataset = np.array(left_dataset)
        right_dataset = np.array(right_dataset)
        return left_dataset, right_dataset

    
    def entropy(self, y):
        entropy = 0
        # find unique values in y
        labels = np.unique(y)
        # loop over each unique value
        for label in labels:
            # find elements in y that are equal to that label
            label_examples = y[y == label]
            # get ratio of the label
            pl = len(label_examples) / len(y)
            # calculate entropy
            entropy += -pl * np.log2(pl)
        return entropy

    
    def information_gain(self, parent, left, right):
        # set initial information gain to 0
        information_gain = 0
        # compute entropy for parent
        parent_entropy = self.entropy(parent)
        # calculate weight for left and right nodes
        weight_left = len(left) / len(parent)
        weight_right= len(right) / len(parent)
        # compute entropy for left and right nodes
        entropy_left, entropy_right = self.entropy(left), self.entropy(right)
        # calculate weighted entropy 
        weighted_entropy = weight_left * entropy_left + weight_right * entropy_right
        # calculate information gain 
        information_gain = parent_entropy - weighted_entropy
        return information_gain

    
    def best_split(self, dataset, num_samples, num_features):
        # dictionary to store the best split values
        best_split = {'gain':- 1, 'feature': None, 'threshold': None}
        # loop over all the features
        for feature_index in range(num_features):
            #get the feature at the current feature_index
            feature_values = dataset[:, feature_index]
            #get unique values of that feature
            thresholds = np.unique(feature_values)
            # loop over all values of the featurre
            for threshold in thresholds:
                # get left and right datasetss
                left_dataset, right_dataset = self.split_data(dataset, feature_index, threshold)
                # check if either datasets is empty
                if len(left_dataset) and len(right_dataset):
                    # get y values of the parent and left, right nodes
                    y, left_y, right_y = dataset[:, -1], left_dataset[:, -1], right_dataset[:, -1]
                    # compute information gain based on the y values
                    information_gain = self.information_gain(y, left_y, right_y)
                    # update the best split if conditions are met
                    if information_gain > best_split["gain"]:
                        best_split["feature"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left_dataset"] = left_dataset
                        best_split["right_dataset"] = right_dataset
                        best_split["gain"] = information_gain
        return best_split

    
    def calculate_leaf_value(self, y):
        #Convert y to list
        y = list(y)
        #get the highest present class in the array
        most_occuring_value = max(y, key=y.count)
        return most_occuring_value

    
    
    def build_tree(self, dataset, current_depth=0):
        # split the dataset into X, y values
        X, y = dataset[:, :-1], dataset[:, -1]
        n_samples, n_features = X.shape
        # keeps spliting until stopping conditions are met
        if n_samples >= self.min_samples and current_depth <= self.max_depth:
            # Get the best split
            best_split = self.best_split(dataset, n_samples, n_features)
            # Check if gain isn't zero
            if best_split["gain"]:
                # continue splitting the left and the right child. Increment current depth
                left_node = self.build_tree(best_split["left_dataset"], current_depth + 1)
                right_node = self.build_tree(best_split["right_dataset"], current_depth + 1)
                # return decision node
                return Node(best_split["feature"], best_split["threshold"],
                            left_node, right_node, best_split["gain"])

        # compute leaf node value
        leaf_value = self.calculate_leaf_value(y)
        # return leaf node value
        return Node(value=leaf_value)

    
    def fit(self, X, y):
        y = y.reshape(y.shape[0], 1)
        dataset = np.concatenate((X, y), axis=1)  
        self.root = self.build_tree(dataset)


    
    def predict(self, X):
        predictions = []
        for x in X:
            prediction = self.make_prediction(x, self.root)
            predictions.append(prediction)
        np.array(predictions)
        return predictions
    
    
    def make_prediction(self, x, node):
        # if the node has value i.e it's a leaf node extract it's value
        if node.value != None: 
            return node.value
        else:
            #if it's node a leaf node we'll get it's feature and traverse through the tree accordingly
            feature = x[node.feature]
            if feature <= node.threshold:
                return self.make_prediction(x, node.left)
            else:
                return self.make_prediction(x, node.right)

In [13]:
from sklearn.tree import DecisionTreeClassifier
m = DecisionTreeClassifier()


# Random Forest

## How the algorithm works

**For specified  number of times we will :**

**Create training sets of the same size using sampling with replacement**

**Train a decision tree on the sampled data**

**Get the votes of the label from each tree and  pick the majority vote**

**RandomForests are called random because each tree  is constructed on a bootstrapped subset of our data**

## Random Forest  Implementation


In [14]:
class RandomForest:
    def __init__(self, n_trees=7, max_depth=7, min_samples=2):
        self.n_trees = n_trees
        self.max_depth=max_depth
        self.min_samples=min_samples
        self.trees = []

    def fit(self, X, y):
        #create an empty list to store the trees
        self.trees = []
        #concatenate X,y into dataset
        dataset = np.concatenate((X, y), axis=1)
        # loop for number of trees
        for _ in range(self.n_trees):
            # make create a tree instance
            tree = DecisionTreeClassifier()
            dataset_sample = self.bootstrap_samples(dataset)
            #get X, y samples from the dataset
            X_sample, y_sample = dataset_sample[:, :-1], dataset_sample[:, -1]
            # fit the tree to the random sample data
            tree.fit(X_sample, y_sample)
            # store the tree in trees list
            self.trees.append(tree)

    def bootstrap_samples(self, dataset):
        # get number of samples
        n_samples = dataset.shape[0]
        # get indexs to index X, y by using sampling with replacement
        np.random.seed(1)
        indexes = np.random.choice(n_samples, n_samples, replace=True)
        return dataset[indexes]

    def most_common_label(self, y):
        y = list(y)
        # get the highest present class in the array
        most_occuring_value = max(y, key=y.count)
        return most_occuring_value


    def predict(self, X):
        #get prediction from each tree in the tree list on the test data
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # get prediction for the same sample from all trees for each sample in the test data
        preds = np.swapaxes(predictions, 0, 1)
        #get the most voted value by the trees and store it in the final predictions array
        majority_predictions = np.array([self.most_common_label(pred) for pred in preds])
        return majority_predictions

<a id="6"></a>
<h1 style='background:#43ae38;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Evaluation</center></h1>
    

# Evaluation


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=41)

### Compare RandomForest to decision tree

In [16]:
model = RandomForest(10,10,2)
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
predictions = model.predict(X_test) #evaluate the model on the test data

# from sklearn.metrics import accuracy_score
# predictions = model.predict(X_test) #evaluate the model on the test data
accuracy_score(y_test, predictions)

0.9333333333333333

In [17]:
dt = DecisionTree(2,10)
dt.fit(X_train, y_train)
predictions = dt.predict(X_test) #evaluate the model on the test data

# predictions = model.predict(X_test) #evaluate the model on the test data
accuracy_score(y_test, predictions)

0.9

<a id="7"></a>
<h1 style='background:#43ae38;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Conclusion</center></h1>
  

# Conclusion

**We were able to implement RandomForest from scratch**

**RandomForest is giving much better accuracy than a single DecisionTree**

**Note: Results may vary between runs**

<a id="8"></a>
<h1 style='background:#43ae38;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Thank You</center></h1>
  

# Thank You


**Thank you for going through this notebook please leave your feedback**
